In [1]:
import pandas as pd
import numpy as np

In [3]:
# Load the CSV data (assuming the data is in a file named 'nutrition_data.csv')
df = pd.read_csv('data/nutrition_data.csv')

In [5]:
# 1. Basic Data Cleaning (same as before)

# Check for and handle missing values
df.fillna(0, inplace=True)

# Convert relevant columns to numeric
numeric_cols = df.columns.drop(['Shrt_Desc', 'GmWt_Desc1', 'GmWt_Desc2'])
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df.fillna(0, inplace=True)


In [6]:

# 2. Create 'Food_Group' based on the first item in 'Shrt_Desc'

df['Food_Group'] = df['Shrt_Desc'].str.split(',').str[0].str.strip()

# 3. Create 'Secondary_group' based on the first two items in 'Shrt_Desc'

df['Secondary_group'] = df['Shrt_Desc'].str.split(',').str[:2].str.join(', ').str.strip()



In [10]:
# 4. Meaningful Aggregate Functions

# Define the aggregate functions to apply
agg_functions = ['min', 'max', 'median', 'std']

# Aggregate for Food Group
food_group_agg = df.groupby('Food_Group')[numeric_cols].agg(agg_functions)
# Flatten the MultiIndex column names
food_group_agg.columns = ['_'.join(col).strip() for col in food_group_agg.columns]
food_group_agg = food_group_agg.reset_index()

# Aggregate for Secondary Group
secondary_group_agg = df.groupby('Secondary_group')[numeric_cols].agg(agg_functions)
# Flatten the MultiIndex column names
secondary_group_agg.columns = ['_'.join(col).strip() for col in secondary_group_agg.columns]
secondary_group_agg = secondary_group_agg.reset_index()

# 5. Count of Items in Each Group

# Count for Food Group
food_group_counts = df.groupby('Food_Group').size().reset_index(name='Item_Count')

# Count for Secondary Group
secondary_group_counts = df.groupby('Secondary_group').size().reset_index(name='Item_Count')

# 6. Merge Counts with Aggregate Data

food_group_agg = pd.merge(food_group_agg, food_group_counts, on='Food_Group')
secondary_group_agg = pd.merge(secondary_group_agg, secondary_group_counts, on='Secondary_group')

# Preview of the Food Group aggregations with counts
print("Food Group Aggregations:")
print(food_group_agg.head())

# Preview of the Secondary Group aggregations with counts
print("\nSecondary Group Aggregations:")
print(secondary_group_agg.head())

Food Group Aggregations:
      Food_Group  Water_g_min  Water_g_max  Water_g_median  Water_g_std  \
0        ABALONE        60.10        74.56           67.33    10.224764   
1        ABIYUCH        79.90        79.90           79.90          NaN   
2        ACEROLA        91.41        91.41           91.41          NaN   
3  ACEROLA JUICE        94.30        94.30           94.30          NaN   
4    ACORN FLOUR         6.00         6.00            6.00          NaN   

   Energ_Kcal_min  Energ_Kcal_max  Energ_Kcal_median  Energ_Kcal_std  \
0             105             189              147.0        59.39697   
1              69              69               69.0             NaN   
2              32              32               32.0             NaN   
3              23              23               23.0             NaN   
4             501             501              501.0             NaN   

   Protein_g_min  ...  Cholestrl_mg_std  GmWt_1_min  GmWt_1_max  \
0          17.10  ...   